In [ ]:
import pandas as pd
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
# from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
# from manual_library.text_processing import tokenize_text,lemmatisasi_text
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
sns.set()

In [ ]:
import pandas as pd
df = pd.read_csv('lemma.csv', delimiter=",")

# Display the DataFrame
display(df)

,full_text,text_clean,class,tokens,lemma
0,@sobat_anies 2. Keliru,keliru,1,['keliru'],keliru
1,@dccomm_gov Padat,padat,0,['padat'],padat
2,@islah_bahrawi @kangdede78 Pakar radikal radik...,pakar radikal radikul bawakata agama dukung ba...,1,"['pakar', 'radikal', 'radikul', 'bawakata', 'a...",pakar radikal radikul bawakata agama dukung ba...
3,@lengkongtweets @Avolanza Emg otak bengkok lu ...,memang otak bengkok kamu panas asal kamu tahu ...,1,"['memang', 'otak', 'bengkok', 'kamu', 'panas',...",memang otak bengkok kamu panas asal kamu tahu ...
4,Puji Anies Baswedan sebagai Pemimpin Masa Depa...,puji baswedan pimpin masa tokoh muhammadiyah o...,2,"['puji', 'baswedan', 'pimpin', 'masa', 'tokoh'...",puji baswedan pimpin masa tokoh muhammadiyah o...
...,...,...,...,...,...
759,CAPRES RASA JUBIRNYA JOKOWI. Ganjar &amp; An...,capres rasa jubirnya bahas kerja bangun daerah...,1,"['capres', 'rasa', 'jubirnya', 'bahas', 'kerja...",capres rasa jubirnya bahas kerja bangun daerah...
760,@zy_zy_its_me Nih dibaca biar Faham. Pak anies...,baca paham khianat beliau komitmen janji beri ...,1,"['baca', 'paham', 'khianat', 'beliau', 'komitm...",baca paham khianat beliau komitmen janji beri ...
761,IRONIS MEMANG. &gt; Gus Dur : Prabowo Orang Pa...,ironis orang ikhlas orang tahu ikhlas allah bu...,1,"['ironis', 'orang', 'ikhlas', 'orang', 'tahu',...",ironis orang ikhlas orang tahu ikhlas allah bu...
762,Sejak 2019 kami Berdara2 Mendukung @prabowo. 🤔...,dara dukung menghianati perjuagan oposisi peng...,1,"['dara', 'dukung', 'menghianati', 'perjuagan',...",dara dukung menghianati perjuagan oposisi peng...


In [ ]:
def rebrand(review):
  if review['class'] == 0:
    review['class']='netral'
  elif review['class']== 1:
    review['class']='negatif'
  else:
    review['class']='positif'
  return review

In [ ]:
data_rebrand = df.apply(rebrand, axis=1)
df = data_rebrand.sample(frac = 1)
#apply clean
df

,full_text,text_clean,class,tokens,lemma
580,@Dennysiregar7 Ganjar dan MMD bagus dua2 nya ...,bagus projo bagi besar dukung ketua umum parta...,netral,"['bagus', 'projo', 'bagi', 'besar', 'dukung', ...",bagus projo besar dukung ketua umum partai tug...
610,Prabowo : mas Anies saya tidak punya apa apa ...,punya siap mati paham harta tembus jadi menhan...,netral,"['punya', 'siap', 'mati', 'paham', 'harta', 't...",punya siap mati paham harta tembus menhan tahu...
101,@ekowboy2 Goblog... Muhammad hatta koq mau dis...,goblog muhammad hatta disamain perlu lebih kun...,negatif,"['goblog', 'muhammad', 'hatta', 'disamain', 'p...",goblog muhammad hatta disamain perlu lebih kun...
510,Kerukunan Beragama: SKB 2 Menteri (Pembatasan ...,rukun agama menteri batas ibadah prihatin ubah...,netral,"['rukun', 'agama', 'menteri', 'batas', 'ibadah...",rukun agama menteri batas ibadah prihatin ubah...
15,@NovalAssegaf mudah mudahan pa Anies terpilih ...,mudah mudah pilih jadi pimpin indonesia erdogan,positif,"['mudah', 'mudah', 'pilih', 'jadi', 'pimpin', ...",mudah mudah pilih pimpin indonesia erdogan
...,...,...,...,...,...
585,Ganjar Pranowo Skakmat Prabowo yang Pernah Jad...,skakmat jadi ketua hkti debat perdana capres l...,netral,"['skakmat', 'jadi', 'ketua', 'hkti', 'debat', ...",skakmat ketua hkti debat perdana capres laksan...
445,Keren nih Anak -Anak Milenial. Yang paling ken...,keren anak anak milenial kena banget anak anak...,netral,"['keren', 'anak', 'anak', 'milenial', 'kena', ...",keren anak anak milenial kena banget anak anak...
757,Pendukung Ganjar adalah kelompok yang paling g...,dukung kelompok senang akur indonesia adem,negatif,"['dukung', 'kelompok', 'senang', 'akur', 'indo...",dukung kelompok senang akur indonesia adem
222,GUS AANG OPTIMIS GANJAR-MAHFUD MENANG SATU PUT...,aang optimis mahfud menang satu putar,positif,"['aang', 'optimis', 'mahfud', 'menang', 'satu'...",aang optimis mahfud menang satu putar


In [ ]:
dataProcesed = df[['text_clean','class', 'tokens']] #mengambil kolom
dataProcesed

,text_clean,class,tokens
580,bagus projo bagi besar dukung ketua umum parta...,netral,"['bagus', 'projo', 'bagi', 'besar', 'dukung', ..."
610,punya siap mati paham harta tembus jadi menhan...,netral,"['punya', 'siap', 'mati', 'paham', 'harta', 't..."
101,goblog muhammad hatta disamain perlu lebih kun...,negatif,"['goblog', 'muhammad', 'hatta', 'disamain', 'p..."
510,rukun agama menteri batas ibadah prihatin ubah...,netral,"['rukun', 'agama', 'menteri', 'batas', 'ibadah..."
15,mudah mudah pilih jadi pimpin indonesia erdogan,positif,"['mudah', 'mudah', 'pilih', 'jadi', 'pimpin', ..."
...,...,...,...
585,skakmat jadi ketua hkti debat perdana capres l...,netral,"['skakmat', 'jadi', 'ketua', 'hkti', 'debat', ..."
445,keren anak anak milenial kena banget anak anak...,netral,"['keren', 'anak', 'anak', 'milenial', 'kena', ..."
757,dukung kelompok senang akur indonesia adem,negatif,"['dukung', 'kelompok', 'senang', 'akur', 'indo..."
222,aang optimis mahfud menang satu putar,positif,"['aang', 'optimis', 'mahfud', 'menang', 'satu'..."


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import string

In [ ]:
dataProcesed.sample(40)
bow_transformer = CountVectorizer().fit(dataProcesed['tokens'])
#vocab
print(len(bow_transformer.vocabulary_))

2387


In [ ]:
tweets_bow = bow_transformer.transform(dataProcesed['tokens']) #Transform data to BoW
print('Shape of Sparse Matrix: ', tweets_bow.shape)
print('Amount of Non-Zero occurences: ', tweets_bow.nnz)

Shape of Sparse Matrix:  (764, 2387)
Amount of Non-Zero occurences:  7687


In [ ]:
sparsity = (100.0 * tweets_bow.nnz / (tweets_bow.shape[0] *tweets_bow.shape[1])) #cek persentase sparse
print('sparsity: {}'.format(round(sparsity)))

sparsity: 0


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(tweets_bow)
tweets_tfidf = tfidf_transformer.transform(tweets_bow)
print(tweets_tfidf.shape)

(764, 2387)


In [ ]:
#Train Naive Bayes Model klasifikasi teks deteksi sentiment
from sklearn.naive_bayes import MultinomialNB
tweets_detect_model = MultinomialNB().fit(tweets_tfidf, dataProcesed['class'])

In [ ]:
#Model Evaluation
predictions = tweets_detect_model.predict(tweets_tfidf)
print(predictions)

['netral' 'netral' 'negatif' 'netral' 'positif' 'positif' 'negatif'
 'positif' 'negatif' 'negatif' 'positif' 'positif' 'netral' 'negatif'
 'negatif' 'negatif' 'netral' 'positif' 'netral' 'negatif' 'positif'
 'netral' 'positif' 'netral' 'negatif' 'positif' 'positif' 'netral'
 'netral' 'netral' 'positif' 'negatif' 'positif' 'netral' 'positif'
 'negatif' 'positif' 'negatif' 'positif' 'negatif' 'netral' 'negatif'
 'positif' 'positif' 'negatif' 'positif' 'positif' 'positif' 'netral'
 'netral' 'negatif' 'negatif' 'netral' 'positif' 'netral' 'netral'
 'negatif' 'positif' 'netral' 'netral' 'netral' 'positif' 'positif'
 'netral' 'negatif' 'negatif' 'netral' 'positif' 'positif' 'negatif'
 'netral' 'negatif' 'negatif' 'negatif' 'negatif' 'positif' 'positif'
 'netral' 'positif' 'negatif' 'netral' 'netral' 'positif' 'netral'
 'positif' 'netral' 'netral' 'negatif' 'netral' 'negatif' 'positif'
 'negatif' 'netral' 'negatif' 'positif' 'negatif' 'negatif' 'negatif'
 'positif' 'negatif' 'positif' 'positi

In [ ]:
from sklearn.metrics import classification_report
print (classification_report(dataProcesed['class'], predictions))

              precision    recall  f1-score   support

     negatif       0.91      0.94      0.93       255
      netral       0.97      0.91      0.93       254
     positif       0.94      0.96      0.95       255

    accuracy                           0.94       764
   macro avg       0.94      0.94      0.94       764
weighted avg       0.94      0.94      0.94       764



In [ ]:
from sklearn.model_selection import train_test_split

tweet_train, tweet_test, text_train, text_test = train_test_split(dataProcesed['text_clean'], dataProcesed['class'], test_size=0.3)

print(len(tweet_train), len(tweet_test), len(tweet_train) + len(tweet_test))

534 230 764


In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer()),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])
pipeline.fit(tweet_train,text_train)

Pipeline(steps=[('bow', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('classifier', MultinomialNB())])

In [ ]:
tweet_test

483                parah orang ngobrol ledek lidah melet
556                                          anak kucing
9      sunggulah harum daun pandan daun buat buat ket...
162    ajak rakyat indonesia masuk dukung mahfud gibr...
591    kali indonesia bakar hutan angin tiup singapur...
                             ...                        
326               sehat mahfud sosok ingat tuanku rakyat
74                                                 zalim
609    sentimen negatif tinggi jatuh versi drone empr...
334      pilih mahfud gibran pilih pdip kuasa mlegedezzz
350    satu lihat brandingnya jelas calon punya brand...
Name: text_clean, Length: 230, dtype: object

In [ ]:
test = tweet_test.iloc[0]
test = 'keliru;;'
pre = pipeline.predict([test])
pre

array(['negatif'], dtype='<U7')

In [ ]:
predictions = pipeline.predict(tweet_test)
#print(predictions)
print(classification_report(predictions,text_test))

              precision    recall  f1-score   support

     negatif       0.67      0.48      0.56        98
      netral       0.39      0.77      0.52        43
     positif       0.71      0.60      0.65        89

    accuracy                           0.58       230
   macro avg       0.59      0.61      0.57       230
weighted avg       0.63      0.58      0.58       230



In [ ]:
import joblib
print(joblib.__version__)
import pickle
# Pipeline
joblib.dump(pipeline, 'model.pkl')
# pickle.dumps(process_text, 'process_text.pkl')
# pickle.dumps(stopword,'stopword.pkl')
# pickle.dumps(pipeline, open('model.pkl','wb'))

1.3.2


['model.pkl']